In [2]:
import numpy as np

Sammy Suliman, last modified 2/28/2023

# Problem 1a

The purpose of the below code is to solve a system of linear equations via LU decomposition. To do this I split the LU algorithm into several helper functions:

The purpose of the partial_pivot function is to swap out the elements along the main diagonal if they are zero with nonzero entries below the diagonal so that it is nonzero. The inputs are U, the matrix that will become upper-triangular (by cancelling out the elements below the diagonal), and j, which is the current column index for which we are trying to cancel out the entries below the diagonal.

In [3]:
def partial_pivot(U, j):
    if U[j][j] == 0:
        for i in range(len(U)):
            if U[i][j] != 0:
                a = U[i]
                U[i] = U[j]
                U[j] = a
                break
    return U

The purpose of the m_creator function is to create a list of elements $m_{ij}$, which we can place under the $j$-th column of an empty lower-traingular matrix $L_j$ and use to cancel out the entries below the diagonal along the $j$-th column, according to the formula $-m_{ij} = \frac{-a_{ij}}{a_{jj}}$. It takes in the as inputs U, the matrix that will become upper-triangular (by cancelling out the elements below the diagonal), and j, which is the current column index for which we are trying to cancel out the entries below the diagonal.

In [4]:
def m_creator(U, j):
    m_list = []
    for i in range(len(U) - (j+1)):
        m_list.append((-1)*U[i+1][j]/U[j][j])
    return m_list

The purpose of the L_creator function is to create the lower-traingular matrices $L_j$ with the $m_{ij}$ entries below the diagonal on the $j$-th column. It takes in as inputs the m-list, the column index $j$, and the original matrix $A$ (to create a square matrix the same size as $A$).

In [5]:
def L_creator(m_list, j, A):
    L = np.identity(len(A))
    m_list = [1] + m_list
    fill_zeros = (len(A) - len(m_list)) * [0]
    if len(fill_zeros) != 0:
        m_list = fill_zeros + m_list
    L[:, j] = m_list
    return L

The purpose of the pivot_finder function is to detect when a diagonal entry is zero, then travel down the column to locate the first nonzero entry and place its index into a list for future pivoting by the partial_pivot function and to record the index of the pivots for the permutation_matrix function.

In [6]:
def pivot_finder(U, j, A):
    pivot_list = []
    for j in range(len(A)):
        if U[j][j] == 0:
            for i in range(len(U)):
                if U[i][j] != 0:
                    pivot_list.append((i,j))
                    break
    return pivot_list

The purpose of the permutation_matrix function is to construct each permutation matrix $P_i$ by recording each pivot of the rows that is done to construct a nonzero diagonal into a single permutation of the identity matrix. It takes in as input the pivot_list from the pivot_finder function, and the original matrix $A$ as a size parameter.

In [7]:
def permutation_matrix(pivot_list, A):
    P = np.identity(len(A))
    for (i,j) in pivot_list:
        a = P[:, i]
        P[:, i] = P[:, j]
        P[:, j] = a
    return P

The purpose of the LU_decomposition function is to perform LU decomposition by iterating through the columns, pivoting out the zero entries along the diagonal, adding the $m_{ij}$ values under the diagonal to create the $L_j$ matrices, multiplying the $L_j$ matrices into our original matrix $A$ to cancel out the values under the diagonal of our matrix $A$ for each column to create our upper-triangular matrix $U$, while multiplying together all the $L_j$ to construct our lower-traingular matrix $L$. The function takes in our original matrix to be decomposed $A$, and outputs a dictionary of our upper-triangular matrix $U$, lower-triangular matrix $L$, and our matrix of permutations $P$.

In [8]:
def LU_decomposition(A):
    U = A
    L = np.identity(len(A))
    for j in range(len(A)):
        U = partial_pivot(U, j)
        m_list = m_creator(U, j)
        L_k = L_creator(m_list, j, A)
        U = np.matmul(L_k, np.array(U))
        L = np.matmul(L_k, L)
    pivot_list = pivot_finder(U, j, A)
    P = permutation_matrix(pivot_list, U)
    return {'U':U, 'L':L, 'P':P}

The purpose of the matrix_solver function is to solve the system of linear equations $Ax = b$ via the LU_decomposition function, by first constructing a vector $y = L \cdot P \cdot b$, then finding $x = U^{-1} \cdot y$. It takes in as input the matrix $A$, and a vector $b$.

In [9]:
def matrix_solver(A, b):
    if np.linalg.det(np.array(A)) == 0:
        return ('ERROR: Matrix is not invertible')
    LU_dict = LU_decomposition(A)
    U = LU_dict['U']
    L = LU_dict['L']
    P = LU_dict['P']
    y = np.matmul(P, b)
    y = np.matmul(L, y)
    U_inv = np.linalg.inv(U)
    x = np.matmul(U_inv, y)
    return x

# Problem 1b

In [10]:
A = [[5, 1, 0, 2, 1], [0, 4, 0, 1, 2], [1, 1, 4, 1, 1], [0, 1, 2, 6, 0], [0, 0, 1, 2, 4]]

In [11]:
b = [1, 2, 3, 4, 5]

In [12]:
matrix_solver(A, b)

array([-0.17083787, -0.06746464,  0.46028292,  0.52448313,  0.8726877 ])

In [13]:
# verifying the algorithm works correctly with numpy built-in matrix solver
np.linalg.solve(A, b)

array([-0.17083787, -0.06746464,  0.46028292,  0.52448313,  0.8726877 ])

# Problem 1c

In [14]:
A = [[5, 1, 0, 2], [0, 4, 0, 8], [1, 1, 4, 2], [0, 1, 2, 2]]

In [15]:
b = [1, 2, 3, 4]

In [16]:
matrix_solver(A, b)

'ERROR: Matrix is not invertible'

Function correctly outputs error message indicating a non-invertible input matrix $A$